In [1]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [ ]:
import tensorflow as tf
import numpy as np 
def network(input_shape, train=True, android=False):

    if android:
        inputs = tf.keras.layers.Input(shape=input_shape)
        inputs_ = tf.keras.layers.Lambda(lambda x: x[:, :, :, :3])(inputs)
        x_0 = tf.keras.layers.Conv2D(filters=24, kernel_size=5, 
                                     strides=2, padding='same',
                                    kernel_initializer='he_normal')(inputs_)

    else :
        inputs = tf.keras.layers.Input(shape=input_shape)
        x_0 = tf.keras.layers.Conv2D(filters=24, kernel_size=5, 
                                     strides=2, padding='same',
                                    kernel_initializer='he_normal')(inputs)

    x_0 = tf.keras.layers.Activation("relu")(x_0)

    # single BlazeBlock phase
    x_1 = singleBlazeBlock(x_0)
    x_2 = singleBlazeBlock(x_1) 
    x_3 = singleBlazeBlock(x_2, strides=2, filters=48)
    x_4 = singleBlazeBlock(x_3, filters=48)
    x_5 = singleBlazeBlock(x_4, filters=48)

    # double BlazeBlock phase
    x_6 = doubleBlazeBlock(x_5, strides=2)
    x_7 = doubleBlazeBlock(x_6)
    x_8 = doubleBlazeBlock(x_7)
    x_9 = doubleBlazeBlock(x_8, strides=2)
    x10 = doubleBlazeBlock(x_9)
    x11 = doubleBlazeBlock(x10)
    
    output = tf.keras.layers.Conv2D(filters=17, kernel_size=1, 
                                     strides=1, padding='same',
                                    kernel_initializer='he_normal')(x11)
    
    conf = tf.keras.layers.Lambda(lambda x : x[:,:,:,:1])(output)
    bbox = tf.keras.layers.Lambda(lambda x : x[:,:,:,1:5])(output)
    kpts = tf.keras.layers.Lambda(lambda x : x[:,:,:,5:])(output)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=[conf, bbox, kpts])
    
    return model

if __name__ == "__main__":
    pass

In [2]:
def singleBlazeBlock(x, filters=24, kernel_size=5, strides=1, padding='same'):

    # depth-wise separable convolution
    x_0 = tf.keras.layers.SeparableConv2D(
        filters=filters,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        use_bias=False,
        kernel_initializer='he_normal')(x)

    # Residual connection
    if strides == 2:
        input_channels = x.shape[-1]
        output_channels = x_0.shape[-1]

        x_ = tf.keras.layers.MaxPooling2D()(x)

        if output_channels - input_channels != 0:

            # channel padding
            x_ = tf.keras.layers.Conv2D(output_channels, 1, kernel_initializer='he_normal')(x_)

        out = tf.keras.layers.Add()([x_0, x_])
        return tf.keras.layers.Activation("relu")(out)

    out = tf.keras.layers.Add()([x_0, x])
    return tf.keras.layers.Activation("relu")(out)


def doubleBlazeBlock(x, filters_1=24, filters_2=96,
                     kernel_size=5, strides=1, padding='same'):

    # depth-wise separable convolution, project
    x_0 = tf.keras.layers.SeparableConv2D(
        filters=filters_1,
        kernel_size=kernel_size,
        strides=strides,
        padding=padding,
        use_bias=False,
        kernel_initializer='he_normal')(x)

    x_1 = tf.keras.layers.Activation("relu")(x_0)

    # depth-wise separable convolution, expand
    x_2 = tf.keras.layers.SeparableConv2D(
        filters=filters_2,
        kernel_size=kernel_size,
        strides=1,
        padding=padding,
        use_bias=False,
        kernel_initializer='he_normal')(x_1)

    # Residual connection
    if strides == 2:
        input_channels = x.shape[-1]
        output_channels = x_2.shape[-1]

        x_ = tf.keras.layers.MaxPooling2D()(x)

        if output_channels - input_channels != 0:

            # channel padding
            x_ = tf.keras.layers.Conv2D(output_channels, 1, kernel_initializer='he_normal')(x_)

        out = tf.keras.layers.Add()([x_2, x_])
        return tf.keras.layers.Activation("relu")(out)

    out = tf.keras.layers.Add()([x_2, x])
    return tf.keras.layers.Activation("relu")(out)